In [ ]:
import os
from pymongo import MongoClient

### Connection to MongoDB

In [ ]:
mongodb_uri = os.environ.get('MONGODB_URI')
database_name = os.environ.get('DATABASE_NAME')

client = MongoClient(mongodb_uri)
database = client[database_name]

#### Connecting to users_votes collection

In [ ]:
users_votes = database['users_votes']  # last updated users votes collection

#### Drop users votes docs with no user info in users collection

In [ ]:
pipeline = [
    {
        '$lookup': {
            'from': 'users',   # last updated users collection
            'localField': 'user', 
            'foreignField': '_id', 
            'as': 'users_votes_users'
        }
    }, {
        '$match': {
            'users_votes_users': {
                '$ne': []
            }
        }
    }, {
        '$project': {
            'users_votes_users': 0
        }
    }, {
        '$out': 'modified_users_votes'     # new collection
    }
]

In [ ]:
try:
    users_votes.aggregate(pipeline)
except Exception as e:
    print(e)